In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
from hera_cal import redcal
import linsolve

In [2]:
Nants = 37

ants = np.loadtxt('antenna_positions_%d.dat'%Nants)
antpos = {k:v for k,v in zip(range(37),ants)}
reds = redcal.get_reds(antpos)

In [3]:
def calibrate(data,gains):
    calib_data = {}
    for (i, j, pol) in data.keys():
        calib_data[(i, j, pol)] = data[(i, j, pol)]/(gains[(i,'Jxx')] * np.conj(gains[(j,'Jxx')]))
    return calib_data

def average_red_vis(data, gains, subreds):
    sub_bl_visib = calibrate(data, gains)
    vis = 0
    for subbl in subreds:
        vis += sub_bl_visib[subbl]/len(subreds)
    return vis

def optimize_red_vis(data, gains, subreds):
    eqs = {}; const = {}
    for (i,j,pol) in subreds:
        eqs['g_%d * g_%d_ * V'%(i,j)] = data[(i,j,pol)]

    for i in range(37):
        const['g_%d'%(i)] = gains[(i,'Jxx')]    

    lps = linsolve.LinearSolver(eqs, **const)
    X = lps.solve()
    return X['V']

In [4]:
subredbls = reds[0:3]
subviskeys = [(0,1,'xx'),(0,4,'xx'),(0,5,'xx')]

In [ ]:
# Simulated Visibilities and Gains

Nsim = 256
true_gains, true_vis, true_data = redcal.sim_red_data(reds, shape=(1, Nsim), gain_scatter=0)

# No Noise, Solution Starting point

In this simple case, both *redcal* and *subredcal* take no steps to converge and result in a chisq of zero as expected.

In [ ]:
sigma = 0

data = {k:v+ sigma*redcal.noise((1, Nsim)) for k,v in true_data.items()}

In [ ]:
# Redcal
startp = {k:v for k,v in true_gains.items()}
startp.update({k:v for k,v in true_vis.items()})

redcalibrator = redcal.RedundantCalibrator(reds)
sol_degen = redcalibrator.omnical(data, startp)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

In [ ]:
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((sol[(i,'Jxx')]*np.conj(sol[(j,'Jxx')])*sol[k])- data[(i,j,pol)])**2
#chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])

In [ ]:
# Subredcal
redcalibrator_sub = redcal.RedundantCalibrator(subredbls)

subsol_degen = {k:v for k,v in true_gains.items()}
subsol_degen.update({k:v for k,v in true_vis.items() if k in subviskeys})

subsol_degen = redcalibrator_sub.omnical(data, subsol_degen) #gain=0.3, #wgts= wgts, 
                                         #check_after= 1, maxiter= 5000)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [ ]:
chisq_sub = 0
for k in subviskeys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data[(i,j,pol)])**2
        
#chisq_sub = chisq_sub/((90-40+2)*sigma**2)
print np.median(chisq_sub[0]), np.mean(chisq_sub[0])

# No Noise, Flat and Logcal Priors

1. Even with no noise, no scatter and flat priors *redcal*  and *subredcal* both fail to converge to a solution consistently. This proves that **omnical cannot** converge to a solution unless the starting point is close to the solution already.

2. With *logcal* determining the starting point, both *redcal* and *subredcal* converge to a solution consistently.

In [ ]:
# Redcal

redcalibrator = redcal.RedundantCalibrator(reds)

# startp = {k:np.ones_like(v) for k,v in true_gains.items()}
# startp.update({k:np.ones_like(v) for k,v in true_vis.items()})
startp = redcalibrator.logcal(data)

sol_degen = redcalibrator.omnical(data, startp)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

In [ ]:
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((sol[(i,'Jxx')]*np.conj(sol[(j,'Jxx')])*sol[k])- data[(i,j,pol)])**2
#chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])

In [ ]:
# Subredcal
redcalibrator_sub = redcal.RedundantCalibrator(subredbls)

# startp = {k:np.ones_like(v) for k,v in true_gains.items()}
# sstartp.update({k:np.ones_like(v) for k,v in true_vis.items() if k in subviskeys})

startp = redcalibrator_sub.logcal(data)

subsol_degen = redcalibrator_sub.omnical(data, startp)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [ ]:
chisq_sub = 0
for k in subviskeys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data[(i,j,pol)])**2
        
#chisq_sub = chisq_sub/((90-40+2)*sigma**2)
print np.median(chisq_sub[0]), np.mean(chisq_sub[0])

# No Noise, Offset Starting Point

Without noise, *omnical*:

1. **Converges** for the solution as the starting point without any steps.
2. **Does not converge** for flat priors as the starting point
3. **Converges** for *logcal* priors by taking the maximum iterations allowed.

Conclusion: The starting point of omnical critically determines if the algorithm can converge to a solution or not. What is the leeway for this starting point? In the absence of noise, to what percentage of the solution should the starting point be evaluated at?

In [ ]:
# Redcal

redcalibrator = redcal.RedundantCalibrator(reds)

startp = {k:v+0.3*np.random.normal(size=np.shape(v)) for k,v in true_gains.items()}      # 30% error
startp.update({k:v+0.3*np.random.normal(size=np.shape(v)) for k,v in true_vis.items()})

sol_degen = redcalibrator.omnical(data, startp)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

In [ ]:
#bins = np.logspace(-5,-2,num=32,base=10)
plt.figure()
for k,g in true_gains.items():
    plt.hist(np.abs(sol[k]-true_gains[k])[0], bins=32, alpha=0.5)

In [ ]:
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((sol[(i,'Jxx')]*np.conj(sol[(j,'Jxx')])*sol[k])- data[(i,j,pol)])**2

print np.median(chisq[0]), np.mean(chisq[0])
print np.where(chisq[0]>1)

In [ ]:
# Subredcal
redcalibrator_sub = redcal.RedundantCalibrator(subredbls)

startp = {k:v+0.1*np.random.normal(size=np.shape(v)) for k,v in true_gains.items()}
startp.update({k:v+0.1*np.random.normal(size=np.shape(v)) for k,v in true_vis.items() if k in subviskeys})

subsol_degen = redcalibrator_sub.omnical(data, startp)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [ ]:
chisq_sub = 0
for k in subviskeys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data[(i,j,pol)])**2
        
print np.median(chisq_sub[0]), np.mean(chisq_sub[0])
print np.where(chisq_sub[0]>1)

In [ ]:
model_vis_sub = {}
for k in true_vis.keys():
    if k in subsol.keys():
        model_vis_sub[k] = subsol[k]
    else:
        subreds = [bls for bls in reds if k in bls][0]
        model_vis_sub[k] = optimize_red_vis(data, subsol, subreds)

chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*model_vis_sub[k])- data[(i,j,pol)])**2

print np.median(chisq[0]), np.mean(chisq[0])

# Low Noise, Solution Starting Point

In [ ]:
sigma = 0.01  # 1% noise, snr = 100

data = {k:v+ sigma*redcal.noise((1, Nsim)) for k,v in true_data.items()}

In [ ]:
# Redcal
startp = {k:v for k,v in true_gains.items()}
startp.update({k:v for k,v in true_vis.items()})

redcalibrator = redcal.RedundantCalibrator(reds)
sol_degen = redcalibrator.omnical(data, startp)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

In [ ]:
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((sol[(i,'Jxx')]*np.conj(sol[(j,'Jxx')])*sol[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])

In [ ]:
# Subredcal
redcalibrator_sub = redcal.RedundantCalibrator(subredbls)

startp = {k:v for k,v in true_gains.items()}
startp.update({k:v for k,v in true_vis.items() if k in subviskeys})

subsol_degen = redcalibrator_sub.omnical(data, startp)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [ ]:
chisq_sub = 0
for k in subviskeys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data[(i,j,pol)])**2
        
chisq_sub = chisq_sub/((90-40+2)*sigma**2)
print np.median(chisq_sub[0]), np.mean(chisq_sub[0])

In [ ]:
model_vis_sub = {}
for k in true_vis.keys():
    if k in subsol.keys():
        model_vis_sub[k] = subsol[k]
    else:
        subreds = [bls for bls in reds if k in bls][0]
        model_vis_sub[k] = optimize_red_vis(data, subsol, subreds)

chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*model_vis_sub[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])

# Low Noise, LogCal

In [ ]:
# Redcal
redcalibrator = redcal.RedundantCalibrator(reds)

startp = redcalibrator.logcal(data)
# startp = {k:np.ones_like(v) for k,v in true_gains.items()}
# startp.update({k:np.ones_like(v) for k,v in true_vis.items()})

sol_degen = redcalibrator.omnical(data, startp)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

In [ ]:
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((sol[(i,'Jxx')]*np.conj(sol[(j,'Jxx')])*sol[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])

In [ ]:
# Subredcal
redcalibrator_sub = redcal.RedundantCalibrator(subredbls)

startp = redcalibrator_sub.logcal(data)
# startp = {k:np.ones_like(v) for k,v in true_gains.items()}
# startp.update({k:np.ones_like(v) for k,v in true_vis.items() if k in subviskeys})

subsol_degen = redcalibrator_sub.omnical(data, startp)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [ ]:
np.where(subsol_degen[0]['chisq'][0] > 1)

In [ ]:
chisq_sub = 0
for k in subviskeys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data[(i,j,pol)])**2
        
chisq_sub = chisq_sub/((90-40+2)*sigma**2)
print np.median(chisq_sub[0]), np.mean(chisq_sub[0])

In [ ]:
model_vis_sub = {}
for k in true_vis.keys():
    if k in subsol.keys():
        model_vis_sub[k] = subsol[k]
    else:
        subreds = [bls for bls in reds if k in bls][0]
        model_vis_sub[k] = optimize_red_vis(data, subsol, subreds)
        
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*model_vis_sub[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])

# Subredcal: Low Noise, Flat Priors on Only Visibilities

Flat priors on only visibilities also does not result in a convergence!

In [ ]:
# Subredcal
redcalibrator_sub = redcal.RedundantCalibrator(subredbls)

subsol_degen = {k:v for k,v in true_gains.items()}
subsol_degen.update({k:np.ones_like(v) for k,v in true_vis.items() if k in subviskeys})

subsol_degen = redcalibrator_sub.omnical(data, subsol_degen, maxiter=300)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [ ]:
np.where(subsol_degen[0]['chisq'][0] > 1)

In [ ]:
chisq_sub = 0
for k in subviskeys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data[(i,j,pol)])**2
        
chisq_sub = chisq_sub/((90-40+2)*sigma**2)
print np.median(chisq_sub[0]), np.mean(chisq_sub[0])
print np.where(chisq_sub[0] > 1.5)

# Low Noise, Gain Scatter, Solution Starting Point

In [5]:
# Simulated Visibilities and Gains

Nsim = 256
true_gains, true_vis, true_data = redcal.sim_red_data(reds, shape=(1, Nsim), gain_scatter=0.1)

In [ ]:
sigma = 0.01  # 1% noise

data = {k:v+ sigma*redcal.noise((1, Nsim)) for k,v in true_data.items()}

In [ ]:
# Redcal
startp = {k:v for k,v in true_gains.items()}
startp.update({k:v for k,v in true_vis.items()})

redcalibrator = redcal.RedundantCalibrator(reds)
sol_degen = redcalibrator.omnical(data, startp)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

In [ ]:
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((sol[(i,'Jxx')]*np.conj(sol[(j,'Jxx')])*sol[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])

In [ ]:
# Subredcal
redcalibrator_sub = redcal.RedundantCalibrator(subredbls)

subsol_degen = {k:v for k,v in true_gains.items()}
subsol_degen.update({k:v for k,v in true_vis.items() if k in subviskeys})

subsol_degen = redcalibrator_sub.omnical(data, subsol_degen)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [ ]:
chisq_sub = 0
for k in subviskeys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data[(i,j,pol)])**2
        
chisq_sub = chisq_sub/((90-40+2)*sigma**2)
print np.median(chisq_sub[0]), np.mean(chisq_sub[0])

In [ ]:
model_vis_sub = {}
for k in true_vis.keys():
    if k in subsol.keys():
        model_vis_sub[k] = subsol[k]
    else:
        subreds = [bls for bls in reds if k in bls][0]
        model_vis_sub[k] = optimize_red_vis(data, subsol, subreds)
        
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*model_vis_sub[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])
print np.where(chisq_sub[0] > 5)

# Gain Scatter, Low Noise, LogCal

In [ ]:
# Redcal
redcalibrator = redcal.RedundantCalibrator(reds)

# startp = {k:v for k,v in true_gains.items()}
# startp.update({k:v for k,v in true_vis.items()})
startp = redcalibrator.logcal(data)

sol_degen = redcalibrator.omnical(data, startp)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

In [ ]:
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((sol[(i,'Jxx')]*np.conj(sol[(j,'Jxx')])*sol[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])
print np.where(chisq[0]>1.3)

In [ ]:
# Subredcal
redcalibrator_sub = redcal.RedundantCalibrator(subredbls)

startp = redcalibrator_sub.logcal(data)

subsol_degen = redcalibrator_sub.omnical(data, startp)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [ ]:
chisq_sub = 0
for k in subviskeys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data[(i,j,pol)])**2
        
chisq_sub = chisq_sub/((90-40+2)*sigma**2)
print np.median(chisq_sub[0]), np.mean(chisq_sub[0])
print np.where(chisq_sub[0]>1.3)

In [ ]:
model_vis_sub = {}
for k in true_vis.keys():
    if k in subsol.keys():
        model_vis_sub[k] = subsol[k]
    else:
        subreds = [bls for bls in reds if k in bls][0]
        model_vis_sub[k] = optimize_red_vis(data, subsol, subreds)
        
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*model_vis_sub[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])
print np.where(chisq_sub[0] > 5)

# Gain Scatter, Low Noise, Slightly offset Gains and Visibilities

In [ ]:
# Redcal
startp = {k: v+0.1*np.random.normal(size=np.shape(v)) for k,v in true_gains.items()}
startp.update({k:v+1*np.random.normal(size=np.shape(v)) for k,v in true_vis.items()})

redcalibrator = redcal.RedundantCalibrator(reds)
sol_degen = redcalibrator.omnical(data, startp)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

In [ ]:
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((sol[(i,'Jxx')]*np.conj(sol[(j,'Jxx')])*sol[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])
np.where(chisq[0]>1.3)

In [ ]:
# Subredcal
redcalibrator_sub = redcal.RedundantCalibrator(subredbls)

subsol_degen = {k:v+0.03*np.random.normal(size=np.shape(v)) for k,v in true_gains.items()}
subsol_degen.update({k:v+0.1*np.random.normal(size=np.shape(v)) for k,v in true_vis.items() if k in subviskeys})

subsol_degen = redcalibrator_sub.omnical(data, subsol_degen)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [ ]:
chisq_sub = 0
for k in subviskeys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data[(i,j,pol)])**2
        
chisq_sub = chisq_sub/((90-40+2)*sigma**2)
print np.median(chisq_sub[0]), np.mean(chisq_sub[0])
np.where(chisq_sub[0]>1.5)

In [ ]:
model_vis_sub = {}
for k in true_vis.keys():
    if k in subsol.keys():
        model_vis_sub[k] = subsol[k]
    else:
        subreds = [bls for bls in reds if k in bls][0]
        model_vis_sub[k] = optimize_red_vis(data, subsol, subreds)
        
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*model_vis_sub[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])
print np.where(chisq_sub[0] > 10)

# Higher Noise, Gain Scatter, Slightly Offset Starting Point

With higher noise in the measured visibilities, *logcal* can no longer give a good starting point for omnical. If hypothetically this starting point can be obtained through a few longer integrations at the beginning of the run it is useful to see if omnical still gives a solution.

*Omnical* converges with a **good starting point**, upto an SNR of 0.5

In [78]:
sigma = 2  # snr = 0.5

data = {k:v+ sigma*redcal.noise((1, Nsim)) for k,v in true_data.items()}

In [81]:
# Redcal
redcalibrator = redcal.RedundantCalibrator(reds)

#startp = redcalibrator.logcal(data)
startp = {k: v+0.01*np.random.normal(size=np.shape(v)) for k,v in true_gains.items()}
startp.update({k:v+sigma*np.random.normal(size=np.shape(v)) for k,v in true_vis.items()})

sol_degen = redcalibrator.omnical(data, startp)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

In [82]:
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((sol[(i,'Jxx')]*np.conj(sol[(j,'Jxx')])*sol[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])
print np.where(chisq[0] > 1.5)

0.9746671450180742 0.9802375748098894
(array([], dtype=int64),)


In [54]:
sigma = 0.5  # snr=2, snr=1.25

data = {k:v+ sigma*redcal.noise((1, Nsim)) for k,v in true_data.items()}

In [55]:
# Subredcal
redcalibrator_sub = redcal.RedundantCalibrator(subredbls)

subsol_degen = {k:v+0.01*np.random.normal(size=np.shape(v)) for k,v in true_gains.items()}
subsol_degen.update({k:v+0.1*np.random.normal(size=np.shape(v)) for k,v in true_vis.items() if k in subviskeys})

subsol_degen = redcalibrator_sub.omnical(data, subsol_degen)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [56]:
chisq_sub = 0
for k in subviskeys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data[(i,j,pol)])**2
        
chisq_sub = chisq_sub/((90-40+2)*sigma**2)
print np.median(chisq_sub[0]), np.mean(chisq_sub[0])
print np.where(chisq_sub[0] > 1.5)

0.9773286253719966 0.9785506922777871
(array([], dtype=int64),)


In [57]:
model_vis_sub = {}
for k in true_vis.keys():
    if k in subsol.keys():
        model_vis_sub[k] = subsol[k]
    else:
        subreds = [bls for bls in reds if k in bls][0]
        model_vis_sub[k] = optimize_red_vis(data, subsol, subreds)
        
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*model_vis_sub[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])
print np.where(chisq_sub[0] > 10)

1.6360362288650432 1.7906602813856891
(array([], dtype=int64),)


## Higher Noise on RedCal 
## Lower Noise on SubRedCal

In [71]:
print 0.8 *np.sqrt(90./666)
print 0.4 *np.sqrt(666/90.)

0.2940858488375231
1.0881176406988355


In [72]:
sigma = 1.10
data = {k:v+ sigma*redcal.noise((1, Nsim)) for k,v in true_data.items()}

# Redcal
redcalibrator = redcal.RedundantCalibrator(reds)

# startp = {k: v+0.01*np.random.normal(size=np.shape(v)) for k,v in true_gains.items()}
# startp.update({k:v+np.random.normal(size=np.shape(v)) for k,v in true_vis.items()})
startp = redcalibrator.logcal(data)

sol_degen = redcalibrator.omnical(data, startp)
sol = redcalibrator.remove_degen(sol_degen[1], degen_sol= true_gains)

In [73]:
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((sol[(i,'Jxx')]*np.conj(sol[(j,'Jxx')])*sol[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])
print np.where(chisq[0] > 1.5)

0.992076703878245 0.9927880985192596
(array([], dtype=int64),)


In [75]:
sigma = 0.4
data = {k:v+ sigma*redcal.noise((1, Nsim)) for k,v in true_data.items()}

# Subredcal
redcalibrator_sub = redcal.RedundantCalibrator(subredbls)

# subsol_degen = {k:v+0.01*np.random.normal(size=np.shape(v)) for k,v in true_gains.items()}
# subsol_degen.update({k:v+0.1*np.random.normal(size=np.shape(v)) for k,v in true_vis.items() if k in subviskeys})
subsol_degen = redcalibrator_sub.logcal(data)

subsol_degen = redcalibrator_sub.omnical(data, subsol_degen)
subsol = redcalibrator_sub.remove_degen(subsol_degen[1], degen_sol= true_gains)

In [76]:
chisq_sub = 0
for k in subviskeys:
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq_sub += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*subsol[k])- data[(i,j,pol)])**2
        
chisq_sub = chisq_sub/((90-40+2)*sigma**2)
print np.median(chisq_sub[0]), np.mean(chisq_sub[0])
print np.where(chisq_sub[0] > 1.5)

0.9750156250368489 0.98300886017823
(array([], dtype=int64),)


In [77]:
model_vis_sub = {}
for k in true_vis.keys():
    if k in subsol.keys():
        model_vis_sub[k] = subsol[k]
    else:
        subreds = [bls for bls in reds if k in bls][0]
        model_vis_sub[k] = optimize_red_vis(data, subsol, subreds)
        
chisq = 0
for k in true_vis.keys():
    subreds = [bls for bls in reds if k in bls][0]
    for (i,j,pol) in subreds:
        chisq += np.abs((subsol[(i,'Jxx')]*np.conj(subsol[(j,'Jxx')])*model_vis_sub[k])- data[(i,j,pol)])**2
chisq = chisq/((666-100+2)*sigma**2)
print np.median(chisq[0]), np.mean(chisq[0])
print np.where(chisq_sub[0] > 10)

1.7054049095963446 1.9319290282613462
(array([], dtype=int64),)
